In [24]:
%load_ext autoreload
%autoreload 2
from module_test.raw_code.optionlib_2.vol.ssvi.model.chain import (
    _load_chain, get_global_config, 
    MarketChainLoader, 
    ChainOutput)
from module_test.raw_code.optionlib_2.vol.ssvi.model.model_utils import params_cache_key
from module_test.raw_code.optionlib_2.vol.ssvi.controller import get_params_cache
from module_test.raw_code.optionlib_2.vol.ssvi.model.ssvi_model import _SSVIModel, SSVIParentModel, EODMarketSSVIModel, SsviTimeseriesEOD
from trade.assets.Stock import Stock
import pandas as pd
pd.options.plotting.backend = "plotly"
get_global_config().vol_type = 'bs'
get_global_config().vol_type

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<VolType.BS: 'bs'>

In [2]:
get_global_config().vol_side='put'

In [3]:
chain = _load_chain("BAX", "2025-10-17")

In [4]:
bax = MarketChainLoader(symbol="BAX", valuation_date="2025-10-17")

In [5]:
model = _SSVIModel(
    chain=chain,
    valuation_date="2025-10-17",
    right="otm"
)

In [6]:
p_model = SSVIParentModel(
    chain=chain,
    valuation_date="2025-10-17",
)

In [7]:
p_model.fit()

In [8]:
p_model._get_or_build("put", fit=True)
p_model.params

{'put': SSVIModelParams{'var0_hat': 0.19692301458252587, 'var_inf_hat': 0.049655547797253466, 'kappa_hat': 0.27404767942036207, 'eta_hat': 1.1322279627012617, 'lambda_hat': -0.7467157525074255, 'rho_hat': -0.17098618803759325, 'atm_loss': 0.0044507332880411995, 'surface_loss': 0.014878475373416303}}

In [9]:
model.fit()


In [10]:
bax.build_chain()

<ChainOutput(root=BAX, valuation_date=2025-10-17 00:00:00)>

In [11]:
bax_model = EODMarketSSVIModel(
    symbol="PG",
    valuation_date="2025-10-17")

In [12]:
bax_model.fit()
get_global_config().vol_side
bax_model.predict(k=[0.9, 1.0, 1.1], exp = ['1m','3m', '6m', '1y'], strike_type='p',)

vol         fwd
strike exp                      
0.9    1m   0.360791  151.960015
       1y   0.276032  153.443111
       3m   0.279890  151.802050
       6m   0.261036  152.194883
1.0    1m   0.186865  151.960015
       1y   0.271322  153.443111
       3m   0.208824  151.802050
       6m   0.235384  152.194883
1.1    1m   0.379057  151.960015
       1y   0.286704  153.443111
       3m   0.299386  151.802050
       6m   0.276963  152.194883

In [ ]:
# SsviTimeseriesEOD.clear_instances(clear_tree=True)
ts = SsviTimeseriesEOD(symbol="T")

In [39]:
get_global_config().vol_side = 'call'
vols_09 = ts.predict(strike_type='p', start_date = '2025-05-01', exp = ['3m'], k = [0.8])
vols_1_3m = ts.predict(strike_type='p', start_date = '2025-05-01', exp = ['3m'], k = [1.])
vols_1_1y = ts.predict(strike_type='p', start_date = '2025-05-01', exp = ['1y'], k = [1.])
vols_09

,,,vol,fwd
Datetime,strike,exp,,
2025-05-01,0.8,3m,0.745214,27.302767
2025-05-02,0.8,3m,0.781544,27.268299
2025-05-05,0.8,3m,0.784406,27.157539
2025-05-06,0.8,3m,0.724251,27.796449
2025-05-07,0.8,3m,0.733047,27.732758
...,...,...,...,...
2025-10-13,0.8,3m,0.545413,25.605625
2025-10-14,0.8,3m,0.554019,26.140837
2025-10-15,0.8,3m,0.508767,26.070328


In [45]:
vols_1_3m.reset_index().plot(y='vol', x='Datetime')

In [46]:
vols_1_1y.reset_index().plot(y='vol', x='Datetime')

In [69]:
df_chain=ts.model_set['2025-05-15'].chain.chain
df_chain[(df_chain['moneyness'].between(0.99, 1.01)) &
          (df_chain['t']>1)][['moneyness', 'vol', 't', 'right']]

,moneyness,vol,t,right
datetime,,,,
2025-05-15,1.008539,0.284200,1.670089,c
2025-05-15,1.008539,0.237459,1.670089,p
2025-05-15,1.008539,0.249082,1.092402,p
2025-05-15,1.008539,0.300947,1.092402,c
